# Data

Loading and appending the Chicago Crime dataset

In [12]:
import pandas as pd
import duckdb
import openai
import time 
import os

Set the path of the CSV files:

In [13]:
path = "./data_raw"

files = [x for x in os.listdir(path = path) if ".csv" in x]

print(files)

['chicago_crime_2022.csv', 'chicago_crime_2023.csv', 'chicago_crime_2021.csv']


In [14]:
chicago_crime = pd.concat((pd.read_csv(path +"/" + f) for f in files), ignore_index=True)

chicago_crime

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,12589893,JF109865,01/11/2022 03:00:00 PM,087XX S KINGSTON AVE,1565,SEX OFFENSE,INDECENT SOLICITATION OF A CHILD,RESIDENCE,False,True,...,7.0,46,17,1194660.0,1847481.0,2022,09/14/2023 03:41:59 PM,41.736409,-87.562410,"(41.736409029, -87.562410309)"
1,12592454,JF113025,01/14/2022 03:55:00 PM,067XX S MORGAN ST,2826,OTHER OFFENSE,HARASSMENT BY ELECTRONIC MEANS,RESIDENCE,False,True,...,16.0,68,26,1170805.0,1860170.0,2022,09/14/2023 03:41:59 PM,41.771782,-87.649437,"(41.771782439, -87.649436929)"
2,12601676,JF124024,01/13/2022 04:00:00 PM,031XX W AUGUSTA BLVD,1752,OFFENSE INVOLVING CHILDREN,AGGRAVATED CRIMINAL SEXUAL ABUSE BY FAMILY MEMBER,RESIDENCE,False,True,...,36.0,23,17,1155171.0,1906486.0,2022,09/14/2023 03:41:59 PM,41.899206,-87.705506,"(41.899206068, -87.705505587)"
3,12785595,JF346553,08/05/2022 09:00:00 PM,072XX S UNIVERSITY AVE,1544,SEX OFFENSE,SEXUAL EXPLOITATION OF A CHILD,APARTMENT,True,False,...,5.0,69,17,1185135.0,1857211.0,2022,09/14/2023 03:41:59 PM,41.763338,-87.597001,"(41.763337967, -87.597001131)"
4,12808281,JF373517,08/14/2022 02:00:00 PM,055XX W ARDMORE AVE,1562,SEX OFFENSE,AGGRAVATED CRIMINAL SEXUAL ABUSE,RESIDENCE,False,False,...,39.0,11,17,1138383.0,1937953.0,2022,09/14/2023 03:41:59 PM,41.985875,-87.766404,"(41.985875279, -87.766403857)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648826,26461,JE455267,11/24/2021 12:51:00 AM,107XX S LANGLEY AVE,0110,HOMICIDE,FIRST DEGREE MURDER,VACANT LOT,False,False,...,9.0,50,01A,1182822.0,1833730.0,2021,09/19/2022 03:41:05 PM,41.698957,-87.606206,"(41.698957409, -87.606205674)"
648827,26041,JE281927,06/28/2021 01:12:00 AM,117XX S LAFLIN ST,0110,HOMICIDE,FIRST DEGREE MURDER,AUTO,False,False,...,34.0,53,01A,1168442.0,1826982.0,2021,09/01/2022 03:42:17 PM,41.680761,-87.659052,"(41.680760863, -87.659051873)"
648828,26238,JE353715,08/29/2021 03:07:00 AM,010XX N LAWNDALE AVE,0110,HOMICIDE,FIRST DEGREE MURDER,STREET,False,False,...,27.0,23,01A,1151525.0,1906643.0,2021,09/19/2022 03:41:05 PM,41.899709,-87.718893,"(41.899709327, -87.718893208)"
648829,26479,JE465230,12/03/2021 08:37:00 PM,000XX W 78TH PL,0110,HOMICIDE,FIRST DEGREE MURDER,PORCH,True,False,...,6.0,69,01A,1177156.0,1852951.0,2021,09/01/2022 03:42:17 PM,41.751832,-87.626374,"(41.751831742, -87.626373808)"


Source: https://platform.openai.com/examples/default-sql-translate

In [15]:
prompt_template = """

Given the following SQL table, your job is to write queries given a user’s request. \n

CREATE TABLE {} ({}) \n

Write a SQL query that returns - {}
"""

In [16]:
def sql_prompt_generator(table_name, col_names, query):
    prompt = prompt_template.format(table_name, col_names, query)
    return prompt


In [17]:
table = "chicago_crime"
col_names = str(list(chicago_crime.columns)).replace('[', '').replace(']', '')
query = "How many cases are from the year 2023?"

p = sql_prompt_generator(table_name = table, col_names = col_names, query = query)

print(p)





Given the following SQL table, your job is to write queries given a user’s request. 


CREATE TABLE chicago_crime ('ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type', 'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat', 'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate', 'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude', 'Location') 


Write a SQL query that returns - How many cases are from the year 2023?



In [18]:
openai.api_key = os.getenv('OPENAI_KEY')

In [47]:
pd.DataFrame(openai.Model.list()["data"])

,id,object,created,owned_by,permission,root,parent
0,text-search-babbage-doc-001,model,1651172509,openai-dev,"[{'id': 'modelperm-s9n5HnzbtVn7kNc5TIZWiCFS', ...",text-search-babbage-doc-001,None
1,gpt-3.5-turbo-16k-0613,model,1685474247,openai,"[{'id': 'modelperm-6TvrqdxTZPTKF0DrBbCYP3sh', ...",gpt-3.5-turbo-16k-0613,None
2,curie-search-query,model,1651172509,openai-dev,"[{'id': 'modelperm-8aqdyZaKtD3MD831mGbqh1MD', ...",curie-search-query,None
3,gpt-3.5-turbo-16k,model,1683758102,openai-internal,"[{'id': 'modelperm-UTqphsmnPe8SNK52msfOjBhP', ...",gpt-3.5-turbo-16k,None
4,text-search-babbage-query-001,model,1651172509,openai-dev,"[{'id': 'modelperm-hXsRH2IK0hXmWxmLRiNTp70t', ...",text-search-babbage-query-001,None
5,babbage,model,1649358449,openai,"[{'id': 'modelperm-h574xGeqWyBeFDDKaoVTC4CO', ...",babbage,None
6,babbage-search-query,model,1651172509,openai-dev,"[{'id': 'modelperm-1zMLcaRlTvYAdpmvvixnTWlF', ...",babbage-search-query,None
7,text-babbage-001,model,1649364043,openai,"[{'id': 'modelperm-YABzYWjC1kS6M2BnI6Fr9vuS', ...",text-babbage-001,None
8,text-similarity-davinci-001,model,1651172505,openai-dev,"[{'id': 'modelperm-C6TT4mQR3bJQEzEuiZlhKM5u', ...",text-similarity-davinci-001,None
9,davinci,model,1649359874,openai,"[{'id': 'modelperm-RcfCH2MkO5NP9C7wx6kdYnIT', ...",davinci,None


In [59]:
response = openai.Completion.create(engine = "text-davinci-003",
                                     prompt= p)

In [61]:
query = response["choices"][0]["text"]
print(query)
duckdb.sql(query).show()


SELECT COUNT(*) FROM chicago_crime WHERE Year = 2023
┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       200932 │
└──────────────┘



In [62]:
response = openai.Completion.create(engine = "text-davinci-003",
                                     prompt= p,
                                     n = 2)
print(response)
query = response["choices"][0]["text"]

{
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-8BrE4ukZyon9KZQqcajEPMKnusH26",
  "object": "text_completion",
  "created": 1697836900,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\nSELECT COUNT(*) FROM chicago_crime WHERE year = 2023",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    },
    {
      "text": "\nSELECT COUNT(*) \nFROM chicago_crime \n",
      "index": 1,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 133,
    "completion_tokens": 32,
    "total_tokens": 165
  }
}


In [63]:
response = openai.Completion.create(engine = "text-davinci-003",
                                     prompt= p,
                                     n = 2,
                                     temperature = 0)
print(response)
query = response["choices"][0]["text"]

{
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-8BrFhvTP7BT3kJoyGbe9lZJUUb5TW",
  "object": "text_completion",
  "created": 1697837001,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\nSELECT COUNT(*) FROM chicago_crime WHERE Year = 2023",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    },
    {
      "text": "\nSELECT COUNT(*) FROM chicago_crime WHERE Year = 2023",
      "index": 1,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 133,
    "completion_tokens": 32,
    "total_tokens": 165
  }
}


In [65]:
response = openai.Completion.create(engine = "text-davinci-003",
                                     prompt= p,
                                     n = 2,
                                     temperature = 2)
print(response)
query = response["choices"][0]["text"]

{
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-8BrGQ7MspIZXBNs9NOdCeyYHHZ4Dg",
  "object": "text_completion",
  "created": 1697837046,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\nSELECT COUNT('ID') \nFROM `insert tableing data",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    },
    {
      "text": "\nSELECT COUNT(*) AS ' Tom Many Cases'\nFROM ch",
      "index": 1,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 133,
    "completion_tokens": 32,
    "total_tokens": 165
  }
}


In [19]:
def lang2sql(api_key, table_name, col_names, query, engine = "text-davinci-003", n = 1, temperature = 0):
    prompt = sql_prompt_generator(table_name = table_name, col_names = col_names, query = query)
    
    openai.api_key = api_key

    response = openai.Completion.create(engine = engine,
                                        prompt= prompt,
                                        n = n,
                                        temperature = temperature)
    return response


In [20]:
table_name = "chicago_crime"
col_names = str(list(chicago_crime.columns)).replace('[', '').replace(']', '')
api_key = os.getenv('OPENAI_KEY')

In [68]:
query = "How many cases are from the year 2023?"

response = lang2sql(api_key = api_key, table_name = table_name, col_names= col_names, query = query)

In [69]:
print(response)
sql_query = response["choices"][0]["text"]
print(sql_query)

duckdb.sql(sql_query).show()

{
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-8BrPcluYY1YPMG1hfwCeof3ukaa9G",
  "object": "text_completion",
  "created": 1697837616,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\nSELECT COUNT(*) FROM chicago_crime WHERE Year = 2023",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 133,
    "completion_tokens": 16,
    "total_tokens": 149
  }
}

SELECT COUNT(*) FROM chicago_crime WHERE Year = 2023
┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       200932 │
└──────────────┘



In [71]:
query = "How many cases ended up with arrest?"

response = lang2sql(api_key = api_key, table_name = table_name, col_names= col_names, query = query)
sql_query = response["choices"][0]["text"]
print(sql_query)


SELECT COUNT(*) 
FROM chicago_crime 



In [21]:
tbl_describe = duckdb.sql("DESCRIBE SELECT * FROM chicago_crime;")
tbl_describe



┌──────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│     column_name      │ column_type │  null   │   key   │ default │  extra  │
│       varchar        │   varchar   │ varchar │ varchar │ varchar │ varchar │
├──────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ ID                   │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ Case Number          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Date                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Block                │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ IUCR                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Primary Type         │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Description          │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Location Description │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Arrest               │ BOOLEAN     │ YES     │ NUL

In [22]:
col_attr = tbl_describe.df()[["column_name", "column_type"]]

col_attr["column_joint"] = col_attr["column_name"] + " " +  col_attr["column_type"]
col_attr

,column_name,column_type,column_joint
0,ID,BIGINT,ID BIGINT
1,Case Number,VARCHAR,Case Number VARCHAR
2,Date,VARCHAR,Date VARCHAR
3,Block,VARCHAR,Block VARCHAR
4,IUCR,VARCHAR,IUCR VARCHAR
5,Primary Type,VARCHAR,Primary Type VARCHAR
6,Description,VARCHAR,Description VARCHAR
7,Location Description,VARCHAR,Location Description VARCHAR
8,Arrest,BOOLEAN,Arrest BOOLEAN
9,Domestic,BOOLEAN,Domestic BOOLEAN


In [23]:
col_names = str(list(col_attr["column_joint"].values)).replace('[', '').replace(']', '').replace('\'', '')
query = "How many cases ended up with arrest?"
prompt = sql_prompt_generator(table_name = table_name, col_names = col_names, query = query)
print(prompt)



Given the following SQL table, your job is to write queries given a user’s request. 


CREATE TABLE chicago_crime (ID BIGINT, Case Number VARCHAR, Date VARCHAR, Block VARCHAR, IUCR VARCHAR, Primary Type VARCHAR, Description VARCHAR, Location Description VARCHAR, Arrest BOOLEAN, Domestic BOOLEAN, Beat BIGINT, District BIGINT, Ward DOUBLE, Community Area BIGINT, FBI Code VARCHAR, X Coordinate DOUBLE, Y Coordinate DOUBLE, Year BIGINT, Updated On VARCHAR, Latitude DOUBLE, Longitude DOUBLE, Location VARCHAR) 


Write a SQL query that returns - How many cases ended up with arrest?



In [78]:

response = lang2sql(api_key = api_key, table_name = table_name, col_names= col_names, query = query)
print(response)
sql_query = response["choices"][0]["text"]
print(sql_query)

{
  "warning": "This model version is deprecated. Migrate before January 4, 2024 to avoid disruption of service. Learn more https://platform.openai.com/docs/deprecations",
  "id": "cmpl-8BrTEXXWPtIzzLOYcjOwV9FAT5Wcw",
  "object": "text_completion",
  "created": 1697837840,
  "model": "text-davinci-003",
  "choices": [
    {
      "text": "\nSELECT COUNT(*) \nFROM chicago_crime \n",
      "index": 0,
      "logprobs": null,
      "finish_reason": "length"
    }
  ],
  "usage": {
    "prompt_tokens": 172,
    "completion_tokens": 16,
    "total_tokens": 188
  }
}

SELECT COUNT(*) 
FROM chicago_crime 



In [30]:
system = """
Given the following SQL table, your job is to write queries given a user’s request. 


CREATE TABLE chicago_crime (ID BIGINT, Case Number VARCHAR, Date VARCHAR, Block VARCHAR, IUCR VARCHAR, Primary Type VARCHAR, Description VARCHAR, Location Description VARCHAR, Arrest BOOLEAN, Domestic BOOLEAN, Beat BIGINT, District BIGINT, Ward DOUBLE, Community Area BIGINT, FBI Code VARCHAR, X Coordinate DOUBLE, Y Coordinate DOUBLE, Year BIGINT, Updated On VARCHAR, Latitude DOUBLE, Longitude DOUBLE, Location VARCHAR) 
"""

prompt = "Write a SQL query that returns - How many cases ended up with arrest?"

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {
      "role": "system",
      "content": system
    },
    {
      "role": "user",
      "content": prompt
    }
  ],
  temperature=1,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [40]:
print(response)

{
  "id": "chatcmpl-8C2XhXlX8XHb9P9olwbNmh7WmwRYY",
  "object": "chat.completion",
  "created": 1697880401,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "SELECT COUNT(*) AS arrest_cases\nFROM chicago_crime\nWHERE Arrest = true;"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 134,
    "completion_tokens": 17,
    "total_tokens": 151
  }
}


In [41]:
sql_query = response["choices"][0]["message"]["content"]
print(sql_query)


SELECT COUNT(*) AS arrest_cases
FROM chicago_crime
WHERE Arrest = true;


In [42]:
duckdb.sql(sql_query).show()

┌──────────────┐
│ arrest_cases │
│    int64     │
├──────────────┤
│        77635 │
└──────────────┘



In [134]:
def create_message(table_name, query):

    class message:
        def __init__(message, system, user, column_names, column_attr):
            message.system = system
            message.user = user
            message.column_names = column_names
            message.column_attr = column_attr

    
    system_template = """

    Given the following SQL table, your job is to write queries given a user’s request. \n

    CREATE TABLE {} ({}) \n
    """

    user_template = "Write a SQL query that returns - {}"
    
    tbl_describe = duckdb.sql("DESCRIBE SELECT * FROM " + table_name +  ";")
    col_attr = tbl_describe.df()[["column_name", "column_type"]]
    col_attr["column_joint"] = col_attr["column_name"] + " " +  col_attr["column_type"]
    col_names = str(list(col_attr["column_joint"].values)).replace('[', '').replace(']', '').replace('\'', '')

    system = system_template.format(table_name, col_names)
    user = user_template.format(query)

    m = message(system = system, user = user, column_names = col_attr["column_name"], column_attr = col_attr["column_type"])
    return m


    

In [135]:
query = "How many cases ended up with arrest?"
msg = create_message(table_name = "chicago_crime", query = query)

print(msg.system)
print(msg.user)
print(msg.column_names)
print(msg.column_attr)



    Given the following SQL table, your job is to write queries given a user’s request. 


    CREATE TABLE chicago_crime (ID BIGINT, Case Number VARCHAR, Date VARCHAR, Block VARCHAR, IUCR VARCHAR, Primary Type VARCHAR, Description VARCHAR, Location Description VARCHAR, Arrest BOOLEAN, Domestic BOOLEAN, Beat BIGINT, District BIGINT, Ward DOUBLE, Community Area BIGINT, FBI Code VARCHAR, X Coordinate DOUBLE, Y Coordinate DOUBLE, Year BIGINT, Updated On VARCHAR, Latitude DOUBLE, Longitude DOUBLE, Location VARCHAR) 

    
Write a SQL query that returns - How many cases ended up with arrest?
0                       ID
1              Case Number
2                     Date
3                    Block
4                     IUCR
5             Primary Type
6              Description
7     Location Description
8                   Arrest
9                 Domestic
10                    Beat
11                District
12                    Ward
13          Community Area
14                FBI Code

In [146]:


def add_quotes(query, col_names):
    for i in col_names:
        if i in query:
            query = str(query).replace(i, '"' + i + '"') 
    return(query)


def lang2sql(api_key, table_name, query, model = "gpt-3.5-turbo", temperature = 0, max_tokens = 256, top_p = 1, frequency_penalty = 0,presence_penalty= 0):
    class response:
        def __init__(output, message, response, sql):
            output.message = message
            output.response = response
            output.sql = sql

    openai.api_key = api_key

    m = create_message(table_name = table_name, query = query)

    message = [
    {
      "role": "system",
      "content": m.system
    },
    {
      "role": "user",
      "content": m.user
    }
    ]
    
    openai_response = openai.ChatCompletion.create(
        model = model,
        messages = message,
        temperature = temperature,
        max_tokens = max_tokens,
        top_p = top_p,
        frequency_penalty = frequency_penalty,
        presence_penalty = presence_penalty)
    
    sql_query = add_quotes(query = openai_response["choices"][0]["message"]["content"], col_names = m.column_names)

    output = response(message = m, response = openai_response, sql = sql_query)

    return output


In [147]:
query = "How many cases ended up with arrest?"
response = lang2sql(api_key = api_key, table_name = "chicago_crime", query = query)

In [148]:
print(response.message)
print(response.sql)
print(response.response)

<__main__.create_message.<locals>.message object at 0xffff4035c2e0>
SELECT COUNT(*) FROM chicago_crime WHERE "Arrest" = true;
{
  "id": "chatcmpl-8CglDSYrYSzUyOZP25MjNGGd8VC6b",
  "object": "chat.completion",
  "created": 1698034999,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "SELECT COUNT(*) FROM chicago_crime WHERE Arrest = true;"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 137,
    "completion_tokens": 12,
    "total_tokens": 149
  }
}


In [149]:
duckdb.sql(response.sql).show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        77635 │
└──────────────┘



In [152]:
query = "How many cases ended up with arrest between 2022 and 2023?"
response = lang2sql(api_key = api_key, table_name = "chicago_crime", query = query)
print(response.sql)

duckdb.sql(response.sql).show()

SELECT COUNT(*) FROM chicago_crime WHERE "Arrest" = TRUE AND "Year" BETWEEN 2022 AND 2023;
┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│        51265 │
└──────────────┘



In [153]:
query = "Summarize the cases by primary type"
response = lang2sql(api_key = api_key, table_name = "chicago_crime", query = query)

print(response.sql)

duckdb.sql(response.sql).show()

SELECT "Primary Type", COUNT(*) as TotalCases
FROM chicago_crime
GROUP BY "Primary Type"
┌────────────────────────────┬────────────┐
│        Primary Type        │ TotalCases │
│          varchar           │   int64    │
├────────────────────────────┼────────────┤
│ THEFT                      │     139021 │
│ ARSON                      │       1344 │
│ KIDNAPPING                 │        319 │
│ HUMAN TRAFFICKING          │         32 │
│ OTHER NARCOTIC VIOLATION   │         11 │
│ GAMBLING                   │         33 │
│ ASSAULT                    │      58685 │
│ BURGLARY                   │      19898 │
│ CRIMINAL DAMAGE            │      75611 │
│ NARCOTICS                  │      13931 │
│     ·                      │         ·  │
│     ·                      │         ·  │
│     ·                      │         ·  │
│ OBSCENITY                  │        127 │
│ BATTERY                    │     115760 │
│ CRIMINAL TRESPASS          │      11255 │
│ CRIMINAL SEXUAL ASSAULT    │ 